In [19]:
import pandas as pd
import numpy as np
import os
import re

# ============================================================
# 🧭 Y14_경찰청 지구대·파출소 수치화 전처리 (법정동코드 기반 시·구·동 파생)
# - 파일 로드
# - 법정동코드로 시·구·동 파생
# - 구·동별 지구대/파출소 수 집계 및 저장
# ============================================================

# ===== 0) 경로 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y14_경찰청_전국 지구대 파출소 주소 현황_20241231_TM.csv"
output_dir  = "/Users/mac/Documents/SORA_Project/data/preprocessing"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "Y14_전처리_경찰청_전국 지구대 파출소 주소 현황_20241231_TM.csv")

# ===== 1) 로드 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
df.columns = df.columns.str.strip()
df.info()

# ===== 2) 법정동코드로 시·구·동 파생 =====
df["법정동코드_10자리"] = (
    df["법정동코드"]
    .fillna(0)
    .astype(np.int64)
    .astype(str)
    .str.zfill(10)
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2045 entries, 0 to 2044
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      2045 non-null   object 
 1   시도청     2024 non-null   object 
 2   경찰서     2024 non-null   object 
 3   관서명     2024 non-null   object 
 4   구분      2024 non-null   object 
 5   전화번호    2024 non-null   object 
 6   주소      2024 non-null   object 
 7   입력주소    2024 non-null   object 
 8   X       2024 non-null   float64
 9   Y       2024 non-null   float64
 10  CLSS    2024 non-null   object 
 11  PNU     2022 non-null   float64
 12  주소구분    2024 non-null   object 
 13  표준신주소   2010 non-null   object 
 14  표준구주소   2024 non-null   object 
 15  우편번호    1960 non-null   float64
 16  행정동코드   1997 non-null   float64
 17  행정동명    1997 non-null   object 
 18  법정동코드   1990 non-null   float64
 19  법정동명    1997 non-null   object 
dtypes: float64(6), object(14)
memory usage: 319.7+ KB


In [23]:
# ===== 2) 법정동코드로 시·구·동 파생 =====
df["법정동코드_10자리"] = (
    df["법정동코드"]
    .fillna(0)
    .astype(np.int64)
    .astype(str)
    .str.zfill(10)
)
df["시도코드"] = df["법정동코드_10자리"].str[:2]
df["시군구코드"] = df["법정동코드_10자리"].str[2:5]

sido_map = {"11": "서울특별시"}
df["시"] = df["시도코드"].map(sido_map)

gu_map = {
    '110': '종로구', '140': '중구', '170': '용산구', '200': '성동구', '215': '광진구',
    '230': '동대문구', '260': '중랑구', '290': '성북구', '305': '강북구', '320': '도봉구',
    '350': '노원구', '380': '은평구', '410': '서대문구', '440': '마포구', '470': '양천구',
    '500': '강서구', '530': '구로구', '545': '금천구', '560': '영등포구', '590': '동작구',
    '620': '관악구', '650': '서초구', '680': '강남구', '710': '송파구', '740': '강동구'
}
df["구"] = df["시군구코드"].map(gu_map)
df["동"] = df["법정동명"] if "법정동명" in df.columns else np.nan

In [24]:
# ===== 3) 서울특별시 필터링 =====
df_seoul = df[df["시"] == "서울특별시"].copy()

# ===== 4) 구·동별 집계 =====
df_grouped = (
    df_seoul.groupby(["구", "동"])
    .size()
    .reset_index(name="동별 지구대/파출소 수")
    .sort_values(by=["구", "동"], ascending=[True, True])
    .reset_index(drop=True)
)
df_grouped = df_grouped.rename(columns={"구": "지역구별", "동": "동별"})


In [26]:
# ===== 5) 각 구별 소계 문자 출력 =====
df_summary = (
    df_grouped.groupby("지역구별")["동별 지구대/파출소 수"]
    .sum()
    .reset_index()
    .sort_values(by="지역구별")
)

summary_text = ", ".join([f"{row['지역구별']} {row['동별 지구대/파출소 수']}개" for _, row in df_summary.iterrows()])
print("\n📢 [구별 소계 요약]")
print(summary_text)

# ===== 6) 결과 확인 =====
print("\n✅ [집계 결과 - 상위 20건]")
print(df_grouped.head(20))



📢 [구별 소계 요약]
강남구 14개, 강동구 8개, 강북구 9개, 강서구 11개, 관악구 8개, 광진구 10개, 구로구 8개, 금천구 5개, 노원구 8개, 도봉구 8개, 동대문구 10개, 동작구 7개, 마포구 7개, 서대문구 8개, 서초구 11개, 성동구 9개, 성북구 10개, 송파구 11개, 양천구 8개, 영등포구 10개, 용산구 4개, 은평구 8개, 종로구 20개, 중구 14개, 중랑구 8개

✅ [집계 결과 - 상위 20건]
   지역구별    동별  동별 지구대/파출소 수
0   강남구   개포동             1
1   강남구   논현동             2
2   강남구   대치동             1
3   강남구   삼성동             2
4   강남구   세곡동             1
5   강남구   수서동             1
6   강남구   신사동             1
7   강남구  압구정동             1
8   강남구   역삼동             2
9   강남구   일원동             1
10  강남구   청담동             1
11  강동구   강일동             1
12  강동구   고덕동             1
13  강동구   둔촌동             2
14  강동구   명일동             1
15  강동구   상일동             1
16  강동구   성내동             1
17  강동구   암사동             1
18  강북구   미아동             3
19  강북구    번동             2


In [28]:
# ===== 7) 파일 저장 =====
output_path = os.path.join(output_dir, "Y14_경찰청_전국 지구대 파출소 주소 현황_20241231_TM.csv")
df_grouped.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"\n💾 집계 결과 저장 완료: {output_path}")


💾 집계 결과 저장 완료: /Users/mac/Documents/SORA_Project/data/preprocessing/Y14_경찰청_전국 지구대 파출소 주소 현황_20241231_TM.csv
